In [22]:
### useful: functions for displaying results from sql queries using pandas
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)

In [26]:
import sqlalchemy
# Creating a connection to the database
# We use SQLite here because the database file is an SQLite .db file
# create_engine is used to establish a connection to the database file
from sqlalchemy import create_engine, inspect
engine = create_engine('sqlite:///chinook.db') # 3 slashes - relative path 4 slashes - absolute path
cur = engine.connect()



Tables in the database: ['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


🌟 Exercise 1 : Open the database
open the database using sqlalchemy module interface. create an engine object in a variable named engine
call the connect() method to obtain a connection and place in a variable named cur
now run the code below to to run reflecton on the database, prepare classes that map to the database and create an orm session :

In [32]:
#then we want to reflect on the database, prepare classes that map to the database and create an orm session :
# you create an instance of MetaData, which is an object that holds information about the database's schema (tables, columns, and relationships).
metadata = sqlalchemy.MetaData()
# reads the database schema from the database engine and populates the MetaData object with the database structure.
metadata.reflect(engine)


from sqlalchemy.ext.automap import automap_base

# used to automatically generate ORM mappings for the tables in the database. Base is the base class for the automatically generated ORM classes
Base = automap_base(metadata=metadata)

# analyzes the MetaData object and creates ORM classes for each table in the database.
Base.prepare()

#now each table in the database will be mapped to a class in SQLAlchemy, which you can use to interact with the data using Python objects.

# also prepare an orm session to interact with db
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()



🌟 Exercise 2 : table names
print out all the table names

In [31]:
metadata.tables.keys()

dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'tracks', 'media_types', 'invoices', 'playlist_track', 'playlists'])

In [35]:
# Listing all the mapped classes after reflection
print(Base.classes.keys())


['albums', 'artists', 'customers', 'media_types', 'playlists', 'invoices', 'employees', 'tracks', 'invoice_items', 'genres']


🌟 Exercise 3 : Tracks
print out the first three tracks in the tracks table

In [39]:
# Reference the tracks table
Tracks = Base.classes.tracks

# Query the first 3 tracks
tracks = session.query(Tracks).limit(3).all()

# Print out the first three tracks
for track in tracks:
    print(f"Track ID: {track.TrackId}, Name: {track.Name}")



Track ID: 1, Name: For Those About To Rock (We Salute You)
Track ID: 2, Name: Balls to the Wall
Track ID: 3, Name: Fast As a Shark


🌟 Exercise 4 : Albums from Tracks
print out the track name and albums title of the first 20 tracks in the tracks table

In [40]:
# First, we need references to both the tracks and albums tables
Tracks = Base.classes.tracks
Albums = Base.classes.albums

# Now, let's query the first 20 tracks along with the corresponding album titles
# We'll join the tracks table with the albums table on AlbumId
query = session.query(Tracks.Name, Albums.Title).join(Albums, Tracks.AlbumId == Albums.AlbumId).limit(20)

# Execute the query and fetch the results
tracks_with_albums = query.all()

# Now, let's print the track name and corresponding album title for each result
for track, album in tracks_with_albums:
    print(f"Track Name: {track}, Album Title: {album}")


Track Name: For Those About To Rock (We Salute You), Album Title: For Those About To Rock We Salute You
Track Name: Put The Finger On You, Album Title: For Those About To Rock We Salute You
Track Name: Let's Get It Up, Album Title: For Those About To Rock We Salute You
Track Name: Inject The Venom, Album Title: For Those About To Rock We Salute You
Track Name: Snowballed, Album Title: For Those About To Rock We Salute You
Track Name: Evil Walks, Album Title: For Those About To Rock We Salute You
Track Name: C.O.D., Album Title: For Those About To Rock We Salute You
Track Name: Breaking The Rules, Album Title: For Those About To Rock We Salute You
Track Name: Night Of The Long Knives, Album Title: For Those About To Rock We Salute You
Track Name: Spellbound, Album Title: For Those About To Rock We Salute You
Track Name: Balls to the Wall, Album Title: Balls to the Wall
Track Name: Fast As a Shark, Album Title: Restless and Wild
Track Name: Restless and Wild, Album Title: Restless and Wi

🌟 Exercise : Tracks sold
print out the first 10 track sales from the invoice_items table
for these first 10 sales, print what are the names of the track sold, and the quantity sold

We will need to join the invoice_items table with the tracks table to get the track names and then print the first 10 sales.

In [41]:
# First, we need references to the invoice_items and tracks tables
InvoiceItems = Base.classes.invoice_items
Tracks = Base.classes.tracks

# Now, let's query the first 10 sales from the invoice_items table and join with the tracks table
query = session.query(Tracks.Name, InvoiceItems.Quantity).join(Tracks, InvoiceItems.TrackId == Tracks.TrackId).limit(10)

# Execute the query and fetch the results
sales_with_tracks = query.all()

# Now, let's print the track name and quantity sold for each sale
for track_name, quantity in sales_with_tracks:
    print(f"Track Name: {track_name}, Quantity Sold: {quantity}")


Track Name: Balls to the Wall, Quantity Sold: 1
Track Name: Restless and Wild, Quantity Sold: 1
Track Name: Put The Finger On You, Quantity Sold: 1
Track Name: Inject The Venom, Quantity Sold: 1
Track Name: Evil Walks, Quantity Sold: 1
Track Name: Breaking The Rules, Quantity Sold: 1
Track Name: Dog Eat Dog, Quantity Sold: 1
Track Name: Overdose, Quantity Sold: 1
Track Name: Love In An Elevator, Quantity Sold: 1
Track Name: Janie's Got A Gun, Quantity Sold: 1


🌟 Exercise 6 : Top tracks sold
print the names of top 10 tracks sold, and how many they times they were sold


Here, we need to find the total number of sales for each track and order the results by the number of sales in descending order to get the top 10 tracks.

In [42]:
from sqlalchemy import func

# Reference the invoice_items and tracks tables
InvoiceItems = Base.classes.invoice_items
Tracks = Base.classes.tracks

# Query to find the top 10 tracks by the number of times they were sold
# We use func.sum to calculate total sales and func.count for how many times a track was sold
query = session.query(Tracks.Name, func.sum(InvoiceItems.Quantity).label('total_sales')).\
    join(Tracks, InvoiceItems.TrackId == Tracks.TrackId).\
    group_by(Tracks.Name).\
    order_by(func.sum(InvoiceItems.Quantity).desc()).\
    limit(10)

# Execute the query and fetch the results
top_tracks = query.all()

# Now, print the top 10 tracks and how many times they were sold
for track_name, total_sales in top_tracks:
    print(f"Track Name: {track_name}, Total Sales: {total_sales}")


Track Name: The Trooper, Total Sales: 5
Track Name: Untitled, Total Sales: 4
Track Name: The Number Of The Beast, Total Sales: 4
Track Name: Sure Know Something, Total Sales: 4
Track Name: Hallowed Be Thy Name, Total Sales: 4
Track Name: Eruption, Total Sales: 4
Track Name: Where Eagles Dare, Total Sales: 3
Track Name: Welcome Home (Sanitarium), Total Sales: 3
Track Name: Sweetest Thing, Total Sales: 3
Track Name: Surrender, Total Sales: 3


🌟 Exercise 7 : Top selling artists
Who are the top 10 highest selling artists?

For this task, we need to:

Join the invoice_items, tracks, albums, and artists tables.
Sum up the total sales for each artist and order the results by total sales in descending order.

In [45]:
# Reference the necessary tables
InvoiceItems = Base.classes.invoice_items
Tracks = Base.classes.tracks
Albums = Base.classes.albums
Artists = Base.classes.artists

# Query to find the top 10 highest-selling artists
query = session.query(Artists.Name, func.sum(InvoiceItems.Quantity).label('total_sales')).\
    join(Tracks, InvoiceItems.TrackId == Tracks.TrackId).\
    join(Albums, Tracks.AlbumId == Albums.AlbumId).\
    join(Artists, Albums.ArtistId == Artists.ArtistId).\
    group_by(Artists.Name).\
    order_by(func.sum(InvoiceItems.Quantity).desc()).\
    limit(10)

# Execute the query and fetch the results
top_artists = query.all()

# Now, print the top 10 highest-selling artists and their total sales
for artist_name, total_sales in top_artists:
    print(f"Artist Name: {artist_name}, Total Sales: {total_sales}")

Artist Name: Iron Maiden, Total Sales: 140
Artist Name: U2, Total Sales: 107
Artist Name: Metallica, Total Sales: 91
Artist Name: Led Zeppelin, Total Sales: 87
Artist Name: Os Paralamas Do Sucesso, Total Sales: 45
Artist Name: Deep Purple, Total Sales: 44
Artist Name: Faith No More, Total Sales: 42
Artist Name: Lost, Total Sales: 41
Artist Name: Eric Clapton, Total Sales: 40
Artist Name: R.E.M., Total Sales: 39
